In [1]:
import pandas as pd
import numpy as np

In [2]:
data = {
    'Key 1': ['ABC123', 'ABC123', 'ABC456', 'ABC789', '', '', ''],
    'Key 2': ['', '', 'ABC456', 'ABC789', 'PQR123', 'PQR123', ''],
    'Key 3': ['', '', '', '', '', '', 'XYZ123'],
    'Col 1': ['Fixed Leg', 'Floating Leg', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value'],
    'Col 2': ['Some value 1', 'Some value 2', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value'],
    'Col 3': ['Some value', 'Some value', 'Some value', 'Some value', 'Fixed Leg', 'Floating Leg', 'Some value'],
    'Col 4': ['Some value', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value'],
    'Col 5': ['Some value', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value', 'Some value']
}

df = pd.DataFrame(data)

In [3]:
df

,Key 1,Key 2,Key 3,Col 1,Col 2,Col 3,Col 4,Col 5
0,ABC123,,,Fixed Leg,Some value 1,Some value,Some value,Some value
1,ABC123,,,Floating Leg,Some value 2,Some value,Some value,Some value
2,ABC456,ABC456,,Some value,Some value,Some value,Some value,Some value
3,ABC789,ABC789,,Some value,Some value,Some value,Some value,Some value
4,,PQR123,,Some value,Some value,Fixed Leg,Some value,Some value
5,,PQR123,,Some value,Some value,Floating Leg,Some value,Some value
6,,,XYZ123,Some value,Some value,Some value,Some value,Some value


In [4]:
def flatten_dataframe(df):
    # Melt the data to long format
    df_melt = df.melt(id_vars=['Key 1', 'Key 2', 'Key 3'], 
                      var_name='variable', 
                      value_name='value')

    # Create a helper column for cumcount
    df_melt['helper'] = df_melt.groupby(['Key 1', 'Key 2', 'Key 3', 'variable']).cumcount()+1

    # Pivot the data to wide format
    df_pivot = df_melt.pivot_table(index=['Key 1', 'Key 2', 'Key 3'], 
                                   columns=['variable', 'helper'], 
                                   values='value', 
                                   aggfunc='first')
    
    # Flatten MultiIndex columns
    df_pivot.columns = [f'{i}_{j}' if j != '' else f'{i}' for i, j in df_pivot.columns]
    
    # Reset the index
    df_pivot = df_pivot.reset_index()

    return df_pivot

In [5]:
flatten_dataframe(df)

,Key 1,Key 2,Key 3,Col 1_1,Col 1_2,Col 2_1,Col 2_2,Col 3_1,Col 3_2,Col 4_1,Col 4_2,Col 5_1,Col 5_2
0,,,XYZ123,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN
1,,PQR123,,Some value,Some value,Some value,Some value,Fixed Leg,Floating Leg,Some value,Some value,Some value,Some value
2,ABC123,,,Fixed Leg,Floating Leg,Some value 1,Some value 2,Some value,Some value,Some value,Some value,Some value,Some value
3,ABC456,ABC456,,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN
4,ABC789,ABC789,,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN,Some value,NaN
